In [ ]:
import digitalhub as dh

In [ ]:
proj = dh.get_or_create_project('family-audit')

In [ ]:
func = proj.new_function(
    name="serve", 
    kind="python", 
    python_version="PYTHON3_10", 
    code_src="git+https://github.com/tn-aixpa/document-classifier",     
    handler="src.serve:serve",
    init_function="init",
    requirements=["numpy<2", "pandas==2.1.4","transformer_engine==1.12.0", "transformer_engine_cu12==1.12.0", "transformers==4.46.3", "torch==2.5.1", "torchmetrics==1.6.0"]
)

In [ ]:
serve_run = func.run(
    action="serve",
    volumes=[{ 
            "volume_type": "persistent_volume_claim", 
            "name": "family-audit", 
            "mount_path": "/files", 
            "spec": { "claim_name": "family-audit" }
        }]
)

In [ ]:
serve_run.refresh().status.state

In [ ]:
service = serve_run.refresh().status.service
service

In [ ]:
inputs = {"text": 'famiglia wifi ', "k": 1}
inputs

In [ ]:
serve_run.invoke(json={"inference_input": inputs}).text